In [4]:
import sys
sys.path.append('C:\Users\vaak\.snap\auxdata\snaphu-v1.4.2_win64\bin')
sys.path.append('C:\\Users\\vaak\\.snap\\snap-python\snappy')


In [5]:
#Need to configure Python to use the SNAP-Python (snappy) interface(https://senbox.atlassian.net/wiki/spaces/SNAP/pages/50855941/Configure+Python+to+use+the+SNAP-Python+snappy+interface)
# Read in unzipped Sentinel-1 SLC products (EW and IW modes)
# Parameters to provide: path, outpath, region_code

import datetime
import time
from snappy import ProductIO
from snappy import HashMap
import os, gc
from snappy import GPF

In [6]:
def do_apply_orbit_file(source):
    print('\tApply orbit file...')
    parameters = HashMap()
    parameters.put('Apply-Orbit-File', True)
    output = GPF.createProduct('Apply-Orbit-File', parameters, source)
    return output
def topsar_split(product,IW):
    print('\tApply TOPSAR Split...')
    parameters = HashMap()
    parameters.put('subswath', IW)
    #parameters.put('selectedPolarisations', 'VV')
    output=GPF.createProduct("TOPSAR-Split", parameters, product)
    return output

def do_thermal_noise_removal(source):
    print('\tThermal noise removal...')
    parameters = HashMap()
    parameters.put('removeThermalNoise', True)
    output = GPF.createProduct('ThermalNoiseRemoval', parameters, source)
    return output

def do_calibration(source, polarization, pols):
    print('\tCalibration...')
    parameters = HashMap()
    parameters.put('outputSigmaBand', True)
    #if polarization == 'DH':
     #   parameters.put('sourceBands', 'Intensity_HH,Intensity_HV')
    #elif polarization == 'DV':
     #   parameters.put('sourceBands', 'Intensity_VH,Intensity_VV')
    #elif polarization == 'SH' or polarization == 'HH':
     #   parameters.put('sourceBands', 'Intensity_HH')
    #elif polarization == 'SV':
     #   parameters.put('sourceBands', 'Intensity_VV')
    #else:
     #   print("different polarization!")
        
    parameters.put('selectedPolarisations', pols)
    parameters.put('outputImageScaleInDb', False)
    output = GPF.createProduct("Calibration", parameters, source)
    return output
def do_speckle_filtering(source):
    print('\tSpeckle filtering...')
    parameters = HashMap()
    parameters.put('filter', 'Lee')
    parameters.put('filterSizeX', 5)
    parameters.put('filterSizeY', 5)
    output = GPF.createProduct('Speckle-Filter', parameters, source)
    return output

def do_terrain_correction(source, proj, downsample):
    print('\tTerrain correction...')
    parameters = HashMap()
    parameters.put('demName', 'GETASSE30')
    parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
    parameters.put('mapProjection', proj)       # comment this line if no need to convert to UTM/WGS84, default is WGS84
    parameters.put('saveProjectedLocalIncidenceAngle', True)
    parameters.put('saveSelectedSourceBand', True)
    parameters.put('pixelSpacingInMeter', 10.0)

    while downsample == 1:                      # downsample: 1 -- need downsample to 40m, 0 -- no need to downsample
        parameters.put('pixelSpacingInMeter', 40.0)
        break
    output = GPF.createProduct('Terrain-Correction', parameters, source)
    return output

def do_subset(source, wkt):
    print('\tSubsetting...')
    parameters = HashMap()
    parameters.put('geoRegion', wkt)
    output = GPF.createProduct('Subset', parameters, source)
    return output

def convert_dB(source):
    print('\converting to db…')
    parameters = HashMap()
    output = GPF.createProduct('LinearToFromdB', parameters, source)
    return output

In [7]:
def main():
    ## All Sentinel-1 data sub folders are located within a super folder (make sure the data is already unzipped and each sub folder name ends with '.SAFE'):
    path = r'C:\Users\vaak\Downloads\S1_GRDs'
    outpath = r'C:\Users\vaak\Downloads\s1_preprocessed'
    if not os.path.exists(outpath):
        os.makedirs(outpath)
    ## well-known-text (WKT) file for subsetting (can be obtained from SNAP by drawing a polygon)
    
    LONMIN=str(09.5417)
    LATMIN=str(59.038)
    LONMAX=str(11.0347)
    LATMAX=str(60.4857)
    
   # 'POLYGON((78.2584 17.5296,78.2584 17.5185,78.2632 17.5169,78.2680 17.5185,78.2684 17.5224,78.2636 17.5294,78.2584 17.5296))'
  
    wkt='POLYGON (('+LONMIN+' '+LATMIN+','+LONMAX+' '+LATMIN+','+LONMAX+' '+LATMAX+','+LONMIN+' '+LATMAX+','+LONMIN+' '+LATMIN+'))'

    
    ## UTM projection parameters
    proj = '''PROJCS["WGS 84 / Auto UTM",GEOGCS["World Geodetic System 1984",
    DATUM["World Geodetic System 1984",SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]],
    AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]],UNIT["degree", 0.017453292519943295],
    AXIS["Geodetic longitude", EAST],AXIS["Geodetic latitude", NORTH]],PROJECTION["Transverse_Mercator"],
    PARAMETER["central_meridian", 9.0],PARAMETER["latitude_of_origin", 0.0],
    PARAMETER["scale_factor", 0.9996],PARAMETER["false_easting", 500000.0],
    PARAMETER["false_northing", 0.0],UNIT["m", 1.0],AXIS["Easting", EAST],AXIS["Northing", NORTH]]'''
    for folder in os.listdir(path):
        gc.enable()
        gc.collect()
        sentinel_1 = ProductIO.readProduct(path + "\\" + folder)
        print(sentinel_1)

        loopstarttime=str(datetime.datetime.now())
        print('Start time:', loopstarttime)
        start_time = time.time()

        print(folder)
        ## Extract mode, product type, and polarizations from filename
        modestamp = folder.split("_")[1]
        productstamp = folder.split("_")[2]
        polstamp = folder.split("_")[3]

        polarization = polstamp[2:4]
        if polarization == 'DV':
            pols = 'VH,VV'
        elif polarization == 'DH':
            pols = 'HH,HV'
        elif polarization == 'SH' or polarization == 'HH':
            pols = 'HH'
        elif polarization == 'SV':
            pols = 'VV'
        else:
            print("Polarization error!")

        ## Start preprocessing:
        IW='IW2'
        #topsarsplit=topsar_split(sentinel_1,IW)
        applyorbit = do_apply_orbit_file(sentinel_1)
        thermaremoved = do_thermal_noise_removal(applyorbit)
        calibrated = do_calibration(thermaremoved, polarization, pols)
        down_filtered = do_speckle_filtering(calibrated)
        del applyorbit
        del thermaremoved
        del calibrated
        ## IW images are downsampled from 10m to 40m (the same resolution as EW images).
        tercorrected = do_terrain_correction(down_filtered, proj, 0)
        subset = do_subset(tercorrected, wkt)
        res_db=convert_dB(subset)

        del down_filtered
        #del tercorrected
        print("Writing...")
        outputname='Sigma0_'+folder[17:25]

        ProductIO.writeProduct(res_db, outpath + '\\' + outputname, 'GeoTIFF')
        
        #ProductIO.writeProduct(subset, outpath + '\\' + folder[:-5], 'GeoTIFF')
        del subset

        print('Done.')
        sentinel_1.dispose()
        sentinel_1.closeIO()
        print("--- %s seconds ---" % (time.time() - start_time))

if __name__== "__main__":
    main()

org.esa.snap.core.datamodel.Product[name=S1A_IW_GRDH_1SDV_20201024T170237_20201024T170302_034941_04131D_3321]
('Start time:', '2020-11-04 21:31:32.790000')
S1A_IW_GRDH_1SDV_20201024T170237_20201024T170302_034941_04131D_3321.zip
	Apply orbit file...
	Thermal noise removal...
	Calibration...
	Speckle filtering...
	Terrain correction...
	Subsetting...
\converting to db…
Writing...
Done.
--- 1352.89700007 seconds ---
org.esa.snap.core.datamodel.Product[name=S1A_IW_GRDH_1SDV_20201026T053147_20201026T053212_034963_0413DD_CD54]
('Start time:', '2020-11-04 21:54:05.909000')
S1A_IW_GRDH_1SDV_20201026T053147_20201026T053212_034963_0413DD_CD54.zip
	Apply orbit file...
	Thermal noise removal...
	Calibration...
	Speckle filtering...
	Terrain correction...
	Subsetting...
\converting to db…
Writing...
Done.
--- 832.807999849 seconds ---
org.esa.snap.core.datamodel.Product[name=S1B_IW_GRDH_1SDV_20201025T165346_20201025T165411_023972_02D908_BABC]
('Start time:', '2020-11-04 22:07:58.885000')
S1B_IW_GRD

In [ ]:
xmin=502955.185  ymin=6555846.46 xmax=603750 ymax=6711784.640

    